# Bring Your Own Data (BYOD) Outlier Detection

In this notebook, we show a simple use-case of our system using [OECD](https://data.oecd.org/) dataset. In the dataset, we detect three different types of outliers:
* Global outliers: values that rarely appear in the real-world data. 
* Local outliers: values that are different from other values in the same attribute. 
* Null outliers: values that have no meaning

 ## Setup
 
 * Setup __HOME__ directory
 * Setup pandas options to display full dataframes

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

__HOME__ = Path("../byod-cleaning-api")

In [2]:
import pandas as pd

# options to display full dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# Input csv file is read into pandas DataFrame
input_file = __HOME__ / "data/oecd_cropped.csv"

df = pd.read_csv(input_file, dtype=str)
df.head(5)

,GDP per capita,Gross national income (GNI) per capita,Household disposable income,Real GDP growth,Net saving rate in household disposable income,Gross fixed capital formation,"Agriculture, forestry, fishing",Industry including energy,Construction,"Trade, repairs, transport, accomm., food services","Information, communication",Finance and insurance,Real estate,"Professional, scientific, support services","Public admin., defence, education, health, social work",Other services (ISIC Rev.4 R - U),Government deficit,General government debt,General government revenues,General government expenditures,Government/compulsory expenditure on health,Voluntary expenditure on health,Public social expenditure,Private social expenditure,Public pension expenditure,Private pension expenditure,Net official development assistance (Aid),Total tax revenue,Tax on personal income,Tax on corporate profits,Taxes on goods and services,Taxes on the average worker,Imports of goods and services,Exports of goods and services,Goods trade balance: exports minus imports of goods,Imports of goods,Exports of goods,Service trade balance: exports minus imports of services,Imports of services,Exports of services,Current account balance of payments,Outward FDI stocks,Inward FDI stocks,Inflows of foreign direct investment,Outflows of foreign direct investment,Inflation rate: all items,Inflation rate: all items non food non energy,Inflation rate: food,Inflation rate: energy,Producer Price Indices (PPI): manufacturing,Long-term interest rates,Purchasing power parities,Exchange rates,Indices of price levels,Total primary energy supply (TPES),TPES per unit of GDP at 2000 prices and PPPs,Renewable energy,Crude oil import prices,Households with access to computers,Households with access to internet,Water abstactions,National fish landings in domestic ports,National fish landings in foreign ports,Aquaculture,Municipal waste total,Municipal waste total per capita,CO2 emissions from fuel combustion,Tertiary attainment in population aged 25-64,"Expenditure per student: primary, 2010 prices","Expenditure per student: secondary, 2010 prices","Expenditure per student: tertiary, 2010 prices",Youths 15-19 not in education nor employment,Youths 20-24 not in education nor employment,Employment rate in population aged 15-24,Employment rate in population aged 25-54,Employment rate in population aged 55-64,Incidence of part-time employment,Self-employment rate: total employment,Self-employment rate: male employment,Self-employment rate: female employment,Unemployment rate: total labour force,Unemployment rate: male labour force,Unemployment rate: female labour force,Long-term unemployment: total unemployed,"Labour compensation per unit labour input, total economy",Average time worked per person in employment,Gross domestic expenditure on R&D,Researchers: full-time equivalent,Total population,Population growth rates,Total fertitity rates,Youth population aged less than 15,Elderly population aged 65 and over,Foreign-born population,Foreign population,Unemployment rate in population of native-born men,Unemployment rate in population of foreign-born men,Unemployment rate in population of native-born women,Unemployment rate in population of foreign-born women,Life expectancy at birth,Life expectancy at birth: men,Life expectancy at birth: women,Infant mortality,"Overweight or obese, % of population aged 15 and over",Suicide Rates,Goods transport,Passenger transport
0,41 450,41 868,-0.5,1.8,6.5,4.2,0.7,17.3,5.7,20.2,4.2,6.3,8.7,12.8,21.8,2.2,-4.2,110.6,50.3,54.5,7.9,2.3,28.7,2.0,10.0,3.6,0.54,43.1,12.2,2.8,10.7,56.1,81.1,81.6,-13.5,337.0,323.5,10.1,95.2,105.3,-1.1,..,..,78 329,46 413,3.5,1.8,2.7,16.9,6.8,4.23,0.83,0.72,110,56.2,0.13,3 119.8,110.50,78.9,76.5,5 297,17,3,0 e,5 003,454,93,35,..,..,15 666,6.1,17.1,26.0,79.4,38.7,18.8,14.3,17.5,10.5,7.1,7.1,7.2,48.3,2.4,1 560,9 575,9.4,11 048,1.2,1.8,17.0,17.2,15.0,10.6,5.7,15.5,6.0,14.6,80.7 |,78.0 |,83.3 |,3.4,..,18.1,50 506 e,138 643
1,42 585,43

## Outlier Detection
---------------------------------
BYOD outlier detection service is deployed at https://bclean.mint.isi.edu/detect. 

The `POST` request takes data as follows:
```json
{
    "table":{
        "column1": ["val1", "val2"],
        "column2": ["val3", "val4"]
    }
}
```

In [4]:
data = df.to_dict(orient="list")

# show one column for example
data["GDP per capita"]

['41 450', '42 585', '43 746', '44 720', '45 739', '47 366', '49 526']

----------------------------------------
The response data has the following form:
```json
{
    "table":{
        "column1": ["[[[val1]]]", "val2"],
        "column2": ["val3", "val4"]
    }
}
```
where `[[[value]]]` denotes the outliers

In [7]:
from requests.auth import HTTPBasicAuth
import requests

auth = HTTPBasicAuth('mint', 'asf12jkj!%&')

response = requests.post("https://bclean.mint.isi.edu/detect", json={"table": data}, auth=auth)
result_df = pd.DataFrame.from_dict(response.json()["table"], orient="index").transpose()

--------------------------------
Outliers are annotated as `[[[value]]]`. For example, in the first column `GDP per capita`, all values are global outliers since the regex pattern `[0-9]+ [0-9]+` rarely appears in real-world data.

In [8]:
# show result in the same column order as original file
result_df[df.columns]

,GDP per capita,Gross national income (GNI) per capita,Household disposable income,Real GDP growth,Net saving rate in household disposable income,Gross fixed capital formation,"Agriculture, forestry, fishing",Industry including energy,Construction,"Trade, repairs, transport, accomm., food services","Information, communication",Finance and insurance,Real estate,"Professional, scientific, support services","Public admin., defence, education, health, social work",Other services (ISIC Rev.4 R - U),Government deficit,General government debt,General government revenues,General government expenditures,Government/compulsory expenditure on health,Voluntary expenditure on health,Public social expenditure,Private social expenditure,Public pension expenditure,Private pension expenditure,Net official development assistance (Aid),Total tax revenue,Tax on personal income,Tax on corporate profits,Taxes on goods and services,Taxes on the average worker,Imports of goods and services,Exports of goods and services,Goods trade balance: exports minus imports of goods,Imports of goods,Exports of goods,Service trade balance: exports minus imports of services,Imports of services,Exports of services,Current account balance of payments,Outward FDI stocks,Inward FDI stocks,Inflows of foreign direct investment,Outflows of foreign direct investment,Inflation rate: all items,Inflation rate: all items non food non energy,Inflation rate: food,Inflation rate: energy,Producer Price Indices (PPI): manufacturing,Long-term interest rates,Purchasing power parities,Exchange rates,Indices of price levels,Total primary energy supply (TPES),TPES per unit of GDP at 2000 prices and PPPs,Renewable energy,Crude oil import prices,Households with access to computers,Households with access to internet,Water abstactions,National fish landings in domestic ports,National fish landings in foreign ports,Aquaculture,Municipal waste total,Municipal waste total per capita,CO2 emissions from fuel combustion,Tertiary attainment in population aged 25-64,"Expenditure per student: primary, 2010 prices","Expenditure per student: secondary, 2010 prices","Expenditure per student: tertiary, 2010 prices",Youths 15-19 not in education nor employment,Youths 20-24 not in education nor employment,Employment rate in population aged 15-24,Employment rate in population aged 25-54,Employment rate in population aged 55-64,Incidence of part-time employment,Self-employment rate: total employment,Self-employment rate: male employment,Self-employment rate: female employment,Unemployment rate: total labour force,Unemployment rate: male labour force,Unemployment rate: female labour force,Long-term unemployment: total unemployed,"Labour compensation per unit labour input, total economy",Average time worked per person in employment,Gross domestic expenditure on R&D,Researchers: full-time equivalent,Total population,Population growth rates,Total fertitity rates,Youth population aged less than 15,Elderly population aged 65 and over,Foreign-born population,Foreign population,Unemployment rate in population of native-born men,Unemployment rate in population of foreign-born men,Unemployment rate in population of native-born women,Unemployment rate in population of foreign-born women,Life expectancy at birth,Life expectancy at birth: men,Life expectancy at birth: women,Infant mortality,"Overweight or obese, % of population aged 15 and over",Suicide Rates,Goods transport,Passenger transport
0,[[[41 450]]],[[[41 868]]],[[[-0.5]]],1.8,6.5,4.2,0.7,17.3,5.7,20.2,4.2,6.3,8.7,12.8,21.8,2.2,[[[-4.2]]],110.6,50.3,54.5,7.9,2.3,28.7,2.0,10.0,3.6,0.54,43.1,12.2,2.8,10.7,56.1,81.1,81.6,[[[-13.5]]],[[[337.0]]],323.5,10.1,95.2,105.3,[[[-1.1]]],[[[ ..]]],[[[ ..]]],[[[78 329]]],[[[46 413]]],3.5,1.8,2.7,16.9,6.8,4.23,0.83,0.72,110,56.2,0.13,[[[3 119.8]]],110.50,78.9,76.5,[[[5 297]]],17,3,0 e,[[[5 003]]],454,93,35,[[[ ..]]],[[[ ..]]],[[[15 666]]],6.1,17.1,26.0,79.4,38.7,18.8,14.3,17.5,10.5,7.1,7.1,7.2,48.3,2.4,[[[1 560]]],[[[9 575]]],